<img style="float: right; margin: 5px 5px 20px 20px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="100px" height="75px"/>

# 003 Deep learning

### Microstructura y sistemas de trading 

> **Evelin Ramirez, Pedro Gael Rayas**

## Resumen
Este proyecto desarrolla una estrategia de trading algorítmico optimizando parámetros críticos (stop loss, take profit, número de acciones y configuraciones de indicadores técnicos: RSI, Bollinger Bands y MACD) a través de al menos 50 pruebas con Optuna. Se simula la estrategia mediante backtesting, evaluando su desempeño a través de métricas financieras como el Sharpe, Sortino y Calmar ratios, además del porcentaje de operaciones ganadoras.

## Introducción 
La estrategia propuesta integra técnicas de análisis técnico y modelado predictivo para tomar decisiones de trading informadas. Se optimizan los hiperparámetros relevantes y se aplican indicadores técnicos para generar señales de compra y venta. El rendimiento de la estrategia se evalúa con métricas de riesgo y rentabilidad, proporcionando una solución robusta para el trading algorítmico.

## Metodología y código

### 0. Importar librerías 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import ta
import logging
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

### 1. Carga de datos 

* Se utiliza el archivo CSV 'aapl_5m_train.csv' proporcionado por el profesor, que contiene datos históricos de las acciones de Apple registrados cada 5 minutos. Los datos abarcan el período comprendido desde el 04 de enero de 2021 a las 14:30:00 hasta el 30 de diciembre de 2022 a las 21:00:00.

In [2]:
# Carga de datos
data = pd.read_csv('aapl_5m_train.csv').dropna()
print(f"Tamaño inicial de data: {len(data)}")

Tamaño inicial de data: 39160


### 2.Implementación y Entrenamiento del Modelo LSTM y Cálculo del Sharpe Ratio
Se normalizan las variables y se generan secuencias temporales a partir de los datos históricos para capturar la dinámica del mercado. Con estas secuencias se entrena un modelo LSTM que predice, mediante clasificación binaria, la dirección del movimiento del precio. Finalmente, se calcula el Sharpe Ratio anualizado a partir de los retornos del portafolio, lo que permite evaluar la rentabilidad ajustada por riesgo de la estrategia.

In [3]:
# Prepare_lstm_data is defined
def prepare_lstm_data(dataset, lookback=20, features=None):
    if features is None:
        features = ["Close", "RSI", "BB", "MACD", "MACD_signal"]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset[features])
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i - lookback:i])
        y.append(1 if scaled_data[i, 0] > scaled_data[i - 1, 0] else 0)  # Binary classification based on Close
    X, y = np.array(X), np.array(y)
    
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, y_train, X_test, y_test, scaler

# Updated train_lstm with Input layer
def train_lstm(X_train, y_train, lookback, units=50):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    return model

# Updated calculate_metrics to handle NaN
def calculate_metrics(portfolio_value, wins, losses):
    returns = pd.Series(portfolio_value).pct_change().dropna()
    mean_ret = returns.mean()
    std_ret = returns.std()
    if std_ret == 0 or np.isnan(mean_ret) or np.isnan(std_ret):
        return {"Sharpe": -float('inf')}  # Penalize invalid cases
    sharpe = (mean_ret / std_ret) * np.sqrt(252)  # Annualized Sharpe Ratio
    return {"Sharpe": sharpe}


### 3. Optimización de indicadores

#### 3.1 Optimización del Indicador RSI

Esta sección se encarga de ajustar los parámetros del RSI junto con otros hiperparámetros críticos de la estrategia (como stop loss, take profit, número de acciones y lookback) utilizando Optuna. Se utiliza un modelo LSTM sencillo para generar señales basadas únicamente en el RSI, y se evalúa la estrategia mediante backtesting, devolviendo el Sharpe ratio como métrica de rendimiento. Finalmente, se ejecuta la optimización con 50 pruebas para encontrar la configuración óptima de los parámetros del RSI.

In [4]:
# Optimization for RSI

# Objective function for RSI
def objective_rsi(trial, data, verbose=False):
    rsi_window = trial.suggest_int("rsi_window", 10, 100)
    rsi_lower = trial.suggest_int("rsi_lower", 5, 35)
    rsi_upper = trial.suggest_int("rsi_upper", 65, 95)
    stop_loss = trial.suggest_float("stop_loss", 0.01, 0.2)
    take_profit = trial.suggest_float("take_profit", 0.01, 0.2)
    n_shares = trial.suggest_categorical("n_shares", [1000, 2000, 3000, 3500, 4000])
    lookback = trial.suggest_int("lookback", 10, 50)

    dataset = data.copy()
    rsi = ta.momentum.RSIIndicator(dataset.Close, window=rsi_window)
    dataset["RSI"] = rsi.rsi()

    X_train, y_train, X_test, y_test, scaler = prepare_lstm_data(dataset.dropna(), lookback=lookback, features=["Close", "RSI"])
    model = train_lstm(X_train, y_train, lookback=lookback)

    scaled_data = scaler.transform(dataset[["Close", "RSI"]])
    X_full = [scaled_data[i - lookback:i] for i in range(lookback, len(scaled_data))]
    X_full = np.array(X_full)
    lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()

    dataset = dataset.iloc[lookback:].reset_index(drop=True)
    dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
    dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    dataset["RSI_BUY"] = dataset["RSI"] < rsi_lower
    dataset["RSI_SELL"] = dataset["RSI"] > rsi_upper

    dataset["BUY_SIGNAL"] = dataset["RSI_BUY"]
    dataset["SELL_SIGNAL"] = dataset["RSI_SELL"]

    dataset = dataset.dropna()
    capital = 1000000
    com = 0.5 / 100
    portfolio_value = [capital]
    active_long_pos = None
    active_short_pos = None
    wins = 0
    losses = 0

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + take_profit),
                    "stop_loss": row.Close * (1 - stop_loss)
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * n_shares * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }

        long_value = row.Close * n_shares if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * n_shares) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    if len(portfolio_value) <= 1:
        return -float('inf')
    metrics = calculate_metrics(portfolio_value, wins, losses)
    return metrics["Sharpe"]


In [ ]:
# Running the optimization
study_rsi = optuna.create_study(direction="maximize")
study_rsi.optimize(lambda trial: objective_rsi(trial, data), n_trials=50)
print("Mejores parámetros RSI:", study_rsi.best_params)

[I 2025-03-27 21:41:50,525] A new study created in memory with name: no-name-a305651c-37dd-4143-94ea-8ff9aa5bf081
[I 2025-03-27 21:41:58,019] Trial 0 finished with value: 0.16129149941362161 and parameters: {'rsi_window': 25, 'rsi_lower': 14, 'rsi_upper': 84, 'stop_loss': 0.08734063547106775, 'take_profit': 0.14147924934531048, 'n_shares': 1000, 'lookback': 14}. Best is trial 0 with value: 0.16129149941362161.
[I 2025-03-27 21:42:07,703] Trial 1 finished with value: 0.1620345387701782 and parameters: {'rsi_window': 17, 'rsi_lower': 11, 'rsi_upper': 81, 'stop_loss': 0.15061845422926032, 'take_profit': 0.17083324721772392, 'n_shares': 4000, 'lookback': 20}. Best is trial 1 with value: 0.1620345387701782.
[I 2025-03-27 21:42:23,135] Trial 2 finished with value: -inf and parameters: {'rsi_window': 97, 'rsi_lower': 27, 'rsi_upper': 86, 'stop_loss': 0.1522449176054695, 'take_profit': 0.07366692619820915, 'n_shares': 3000, 'lookback': 36}. Best is trial 1 with value: 0.1620345387701782.
[I 20

#### 3.2 Optimización del Indicador Bollinger bands

Esta sección ajusta los parámetros específicos de las Bollinger Bands (ventana y desviación) junto con hiperparámetros generales de trading (stop loss, take profit, número de acciones y lookback) mediante Optuna. Se genera un modelo LSTM sencillo utilizando únicamente las características "Close" y "BB" para producir señales basadas en las Bollinger Bands. Estas señales se emplean en un backtesting que simula operaciones de compra y venta, actualizando el capital y evaluando el desempeño de la estrategia a través del Sharpe Ratio. Finalmente, se optimizan los parámetros con 50 pruebas para identificar la configuración que maximiza el rendimiento ajustado por riesgo.

In [ ]:
# Objective function for Bollinger Bands
def objective_bb(trial, data, verbose=False):
    # Hiperparámetros específicos de BB
    bb_window = trial.suggest_int("bb_window", 10, 30)
    bb_window_dev = trial.suggest_float("bb_window_dev", 1.5, 3.0)
    
    # Hiperparámetros generales de trading
    stop_loss = trial.suggest_float("stop_loss", 0.01, 0.2)
    take_profit = trial.suggest_float("take_profit", 0.01, 0.2)
    n_shares = trial.suggest_categorical("n_shares", [1000, 2000, 3000, 3500, 4000])
    lookback = trial.suggest_int("lookback", 10, 50)

    dataset = data.copy()
    bb = ta.volatility.BollingerBands(dataset.Close, window=bb_window, window_dev=bb_window_dev)
    dataset["BB"] = bb.bollinger_mavg()
    dataset["BB_lower"] = bb.bollinger_lband()
    dataset["BB_upper"] = bb.bollinger_hband()

    X_train, y_train, X_test, y_test, scaler = prepare_lstm_data(dataset.dropna(), lookback=lookback, features=["Close", "BB"])
    model = train_lstm(X_train, y_train, lookback=lookback)

    scaled_data = scaler.transform(dataset[["Close", "BB"]])
    X_full = [scaled_data[i - lookback:i] for i in range(lookback, len(scaled_data))]
    X_full = np.array(X_full)
    lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()

    dataset = dataset.iloc[lookback:].reset_index(drop=True)
    dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
    dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    dataset["BB_BUY"] = dataset.Close < dataset["BB_lower"]
    dataset["BB_SELL"] = dataset.Close > dataset["BB_upper"]

    dataset["BUY_SIGNAL"] = dataset["BB_BUY"]
    dataset["SELL_SIGNAL"] = dataset["BB_SELL"]

    dataset = dataset.dropna()
    capital = 1000000
    com = 0.5 / 100
    portfolio_value = [capital]
    active_long_pos = None
    active_short_pos = None
    wins = 0
    losses = 0

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + take_profit),
                    "stop_loss": row.Close * (1 - stop_loss)
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * n_shares * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }

        long_value = row.Close * n_shares if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * n_shares) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    if len(portfolio_value) <= 1:
        return -float('inf')
    metrics = calculate_metrics(portfolio_value, wins, losses)
    return metrics["Sharpe"]

In [ ]:
# Run the optimization
study_bb = optuna.create_study(direction="maximize")
study_bb.optimize(lambda trial: objective_bb(trial, data), n_trials=50)
print("Mejores parámetros BB:", study_bb.best_params)

#### 3.3 Optimización del Indicador MACD

Esta sección ajusta los parámetros específicos del MACD (ventana lenta, rápida y de señal) junto con hiperparámetros generales de trading (stop loss, take profit, número de acciones y lookback) mediante Optuna. Se utiliza un modelo LSTM simple, entrenado sobre las características "Close", "MACD" y "MACD_signal", para generar señales de trading basadas en el MACD. Estas señales se emplean en un backtesting que simula operaciones (compras y ventas) y actualiza el capital, evaluando el desempeño de la estrategia a través del Sharpe Ratio. Se realizan 50 pruebas para identificar la configuración que maximiza el rendimiento ajustado por riesgo.

In [ ]:
# Optimization for MACD

# Objective function for MACD
def objective_macd(trial, data, verbose=False):
    # Hiperparámetros específicos de MACD
    macd_window_slow = trial.suggest_int("macd_window_slow", 20, 40)
    macd_window_fast = trial.suggest_int("macd_window_fast", 5, 20)
    macd_window_sign = trial.suggest_int("macd_window_sign", 5, 15)
    
    # Hiperparámetros generales de trading
    stop_loss = trial.suggest_float("stop_loss", 0.01, 0.2)
    take_profit = trial.suggest_float("take_profit", 0.01, 0.2)
    n_shares = trial.suggest_categorical("n_shares", [1000, 2000, 3000, 3500, 4000])
    lookback = trial.suggest_int("lookback", 10, 50)

    dataset = data.copy()
    macd = ta.trend.MACD(dataset.Close, window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_sign)
    dataset["MACD"] = macd.macd()
    dataset["MACD_signal"] = macd.macd_signal()

    X_train, y_train, X_test, y_test, scaler = prepare_lstm_data(dataset.dropna(), lookback=lookback, features=["Close", "MACD", "MACD_signal"])
    model = train_lstm(X_train, y_train, lookback=lookback)

    scaled_data = scaler.transform(dataset[["Close", "MACD", "MACD_signal"]])
    X_full = [scaled_data[i - lookback:i] for i in range(lookback, len(scaled_data))]
    X_full = np.array(X_full)
    lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()

    dataset = dataset.iloc[lookback:].reset_index(drop=True)
    dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
    dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    dataset["MACD_BUY"] = dataset["MACD"] > dataset["MACD_signal"]
    dataset["MACD_SELL"] = dataset["MACD"] < dataset["MACD_signal"]

    dataset["BUY_SIGNAL"] = dataset["MACD_BUY"]
    dataset["SELL_SIGNAL"] = dataset["MACD_SELL"]

    dataset = dataset.dropna()
    capital = 1000000
    com = 0.5 / 100
    portfolio_value = [capital]
    active_long_pos = None
    active_short_pos = None
    wins = 0
    losses = 0

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + take_profit),
                    "stop_loss": row.Close * (1 - stop_loss)
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * n_shares * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }

        long_value = row.Close * n_shares if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * n_shares) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    if len(portfolio_value) <= 1:
        return -float('inf')
    metrics = calculate_metrics(portfolio_value, wins, losses)
    return metrics["Sharpe"]


In [ ]:
# Run the optimization
study_macd = optuna.create_study(direction="maximize")
study_macd.optimize(lambda trial: objective_macd(trial, data), n_trials=50)
print("Mejores parámetros MACD:", study_macd.best_params)

### 4. Evaluación de Estrategias y Simulación de Trading
En esta parte del código se establecen los mejores parámetros para cada indicador (RSI, Bollinger Bands y MACD) y se definen las funciones de soporte para:
* Preprocesar el dataset y calcular los indicadores técnicos.
Entrenar un modelo LSTM (cuando es necesario, como en el caso de MACD) para generar señales de trading.
* Simular la ejecución de la estrategia de trading mediante backtesting, actualizando el capital en función de las operaciones (compras/ventas) y registrando el rendimiento.
* Calcular las métricas de desempeño (como Sharpe, Sortino, Calmar y Win/Loss Percentage).

Posteriormente, se evalúan de forma individual las estrategias basadas en RSI, Bollinger Bands y MACD utilizando los parámetros óptimos predefinidos. Finalmente, se muestran los resultados para cada estrategia, lo que permite comparar el rendimiento y determinar la más efectiva para la estrategia de trading algorítmico.

In [ ]:
# Best parameters from your input
RSI_PARAMS = {
    'rsi_window': 46,
    'rsi_lower': 13,
    'rsi_upper': 65,
    'stop_loss': 0.06425843746848879,
    'take_profit': 0.02250238758183155,
    'n_shares': 1000,
    'lookback': 20  # Default value since not provided
}

BB_PARAMS = {
    'bb_window': 26,
    'bb_window_dev': 1.6419616128479513,
    'stop_loss': 0.020183998738358935,
    'take_profit': 0.03515466268130313,
    'n_shares': 1000,
    'lookback': 20  # Default value since not provided
}

MACD_PARAMS = {
    'macd_window_slow': 25,
    'macd_window_fast': 20,
    'macd_window_sign': 9,
    'stop_loss': 0.011148111728088396,
    'take_profit': 0.012560961178120339,
    'n_shares': 3000,
    'lookback': 20
}

# Supporting functions
def prepare_lstm_data(dataset, lookback=20, features=None):
    if features is None:
        features = ["Close", "RSI", "BB", "MACD", "MACD_signal"]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset[features])
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i - lookback:i])
        y.append(1 if scaled_data[i, 0] > scaled_data[i - 1, 0] else 0)
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test, scaler

def train_lstm(X_train, y_train, lookback, units=50):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    return model

def calculate_metrics(portfolio_value, wins, losses):
    returns = pd.Series(portfolio_value).pct_change().dropna()
    mean_ret = returns.mean()
    std_ret = returns.std()
    downside_std = returns[returns < 0].std() if len(returns[returns < 0]) > 0 else 0
    max_drawdown = (pd.Series(portfolio_value).cummax() - portfolio_value).max() / pd.Series(portfolio_value).cummax().max()

    sharpe = (mean_ret / std_ret) * np.sqrt(252) if std_ret > 0 else -float('inf')
    sortino = (mean_ret / downside_std) * np.sqrt(252) if downside_std > 0 else -float('inf')
    calmar = (mean_ret * 252) / max_drawdown if max_drawdown > 0 else -float('inf')
    win_loss_pct = (wins / (wins + losses) * 100) if (wins + losses) > 0 else 0

    return {
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Win_Loss_Percentage": win_loss_pct,
        "Wins": wins,
        "Losses": losses
    }

# Preprocessing function
def preprocess(data: pd.DataFrame, rsi_window: int, bb_window: int, bb_window_dev: float, 
               macd_window_slow: int, macd_window_fast: int, macd_window_sign: int) -> pd.DataFrame:
    dataset = data.copy()
    
    # RSI
    rsi = ta.momentum.RSIIndicator(dataset.Close, window=rsi_window)
    dataset['RSI'] = rsi.rsi()
    
    # Bollinger Bands
    bb = ta.volatility.BollingerBands(dataset.Close, window=bb_window, window_dev=bb_window_dev)
    dataset['BB'] = bb.bollinger_mavg()
    dataset['BB_LOWER'] = bb.bollinger_lband()
    dataset['BB_UPPER'] = bb.bollinger_hband()
    
    # MACD
    macd = ta.trend.MACD(dataset.Close, window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_sign)
    dataset['MACD'] = macd.macd()
    dataset['MACD_signal'] = macd.macd_signal()

    dataset = dataset.dropna()
    return dataset

# Trading simulation function
def simulate_trading(dataset: pd.DataFrame, strategy: str, stop_loss: float, take_profit: float, n_shares: int, lookback: int = None) -> dict:
    if strategy == 'MACD' and lookback is not None:
        X_train, y_train, _, _, scaler = prepare_lstm_data(dataset.dropna(), lookback=lookback, features=["Close", "MACD", "MACD_signal"])
        model = train_lstm(X_train, y_train, lookback=lookback)
        scaled_data = scaler.transform(dataset[["Close", "MACD", "MACD_signal"]])
        X_full = [scaled_data[i - lookback:i] for i in range(lookback, len(scaled_data))]
        X_full = np.array(X_full)
        lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()
        dataset = dataset.iloc[lookback:].reset_index(drop=True)
        dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
        dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    # Define signals
    if strategy == 'RSI':
        dataset['BUY_SIGNAL'] = dataset['RSI'] < RSI_PARAMS['rsi_lower']
        dataset['SELL_SIGNAL'] = dataset['RSI'] > RSI_PARAMS['rsi_upper']
    elif strategy == 'BB':
        dataset['BUY_SIGNAL'] = dataset.Close < dataset['BB_LOWER']
        dataset['SELL_SIGNAL'] = dataset.Close > dataset['BB_UPPER']
    elif strategy == 'MACD':
        dataset['BUY_SIGNAL'] = dataset['MACD'] > dataset['MACD_signal']
        dataset['SELL_SIGNAL'] = dataset['MACD'] < dataset['MACD_signal']
    else:
        raise ValueError("Strategy must be 'RSI', 'BB', or 'MACD'")

    dataset = dataset.dropna()
    capital = 1000000
    com = 0.125 / 100
    portfolio_value = [capital]
    active_long_pos = None
    active_short_pos = None
    wins = 0
    losses = 0

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + take_profit),
                    "stop_loss": row.Close * (1 - stop_loss)
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * n_shares * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }

        long_value = row.Close * n_shares if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * n_shares) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    return calculate_metrics(portfolio_value, wins, losses)

# Evaluate strategies function
def evaluate_strategies(data: pd.DataFrame):
    # Preprocess data once with all indicators
    dataset = preprocess(
        data,
        rsi_window=RSI_PARAMS['rsi_window'],
        bb_window=BB_PARAMS['bb_window'],
        bb_window_dev=BB_PARAMS['bb_window_dev'],
        macd_window_slow=MACD_PARAMS['macd_window_slow'],
        macd_window_fast=MACD_PARAMS['macd_window_fast'],
        macd_window_sign=MACD_PARAMS['macd_window_sign']
    )

    # Evaluate RSI strategy
    rsi_metrics = simulate_trading(
        dataset,
        strategy='RSI',
        stop_loss=RSI_PARAMS['stop_loss'],
        take_profit=RSI_PARAMS['take_profit'],
        n_shares=RSI_PARAMS['n_shares']
    )

    # Evaluate BB strategy
    bb_metrics = simulate_trading(
        dataset,
        strategy='BB',
        stop_loss=BB_PARAMS['stop_loss'],
        take_profit=BB_PARAMS['take_profit'],
        n_shares=BB_PARAMS['n_shares']
    )

    # Evaluate MACD strategy
    macd_metrics = simulate_trading(
        dataset,
        strategy='MACD',
        stop_loss=MACD_PARAMS['stop_loss'],
        take_profit=MACD_PARAMS['take_profit'],
        n_shares=MACD_PARAMS['n_shares'],
        lookback=MACD_PARAMS['lookback']
    )

    # Print results
    print("RSI Strategy Metrics:", rsi_metrics)
    print("BB Strategy Metrics:", bb_metrics)
    print("MACD Strategy Metrics:", macd_metrics)

# Prepare and run the backtest
# Example: Load your data (replace with your actual data source)
dates = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")
data = pd.DataFrame({
    "Datetime": dates,
    "Close": np.random.normal(100, 10, len(dates))  # Random prices for demo
})

# Ensure data has the required columns
assert "Close" in data.columns and "Datetime" in data.columns, "DataFrame must have 'Close' and 'Datetime' columns"

# Run the evaluation
evaluate_strategies(data)

### 5. Configuración de Parámetros Óptimos y Evaluación de Estrategias

Esta parte del código define manualmente los parámetros óptimos para cada indicador (RSI, Bollinger Bands y MACD) mediante diccionarios y establece funciones de soporte para:

* Preparación de Datos y Entrenamiento del Modelo:
Se normalizan las características y se generan secuencias temporales para entrenar un modelo LSTM que ayuda a generar señales de trading.

* Cálculo de Métricas:
Se calcula el rendimiento de la estrategia (medido con Sharpe, Sortino, Calmar, etc.) a partir de la evolución del portafolio durante el backtesting.

* Preprocesamiento y Simulación de Trading:
Se preprocesa el dataset para calcular los indicadores técnicos con los parámetros óptimos y se simula la estrategia de trading mediante backtesting, actualizando el capital de acuerdo a las señales generadas.

* Evaluación de Estrategias:
Se ejecutan backtests para cada estrategia individual (RSI, BB y MACD) y para una estrategia combinada, imprimiendo las métricas de desempeño para comparar su efectividad.

Esta estructura permite aplicar los parámetros predefinidos para evaluar de forma directa el desempeño de cada estrategia de trading.

In [ ]:
# Best parameters from your input
RSI_PARAMS = {
    'rsi_window': 46,
    'rsi_lower': 13,
    'rsi_upper': 65,
    'stop_loss': 0.06425843746848879,
    'take_profit': 0.02250238758183155,
    'n_shares': 1000,
    'lookback': 20  # Default value since not provided; adjust if needed
}

BB_PARAMS = {
    'bb_window': 26,
    'bb_window_dev': 1.6419616128479513,
    'stop_loss': 0.020183998738358935,
    'take_profit': 0.03515466268130313,
    'n_shares': 1000,
    'lookback': 20  # Default value since not provided; adjust if needed
}

MACD_PARAMS = {
    'macd_window_slow': 25,
    'macd_window_fast': 20,
    'macd_window_sign': 9,
    'stop_loss': 0.011148111728088396,
    'take_profit': 0.012560961178120339,
    'n_shares': 3000,
    'lookback': 20
}

# Supporting functions
def prepare_lstm_data(dataset, lookback=20, features=None):
    if features is None:
        features = ["Close", "RSI", "BB", "MACD", "MACD_signal"]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset[features])
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i - lookback:i])
        y.append(1 if scaled_data[i, 0] > scaled_data[i - 1, 0] else 0)
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test, scaler

def train_lstm(X_train, y_train, lookback, units=50):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    return model

def calculate_metrics(portfolio_value, wins, losses):
    returns = pd.Series(portfolio_value).pct_change().dropna()
    mean_ret = returns.mean()
    std_ret = returns.std()
    downside_std = returns[returns < 0].std() if len(returns[returns < 0]) > 0 else 0
    max_drawdown = (pd.Series(portfolio_value).cummax() - portfolio_value).max() / pd.Series(portfolio_value).cummax().max()

    sharpe = (mean_ret / std_ret) * np.sqrt(252) if std_ret > 0 else -float('inf')
    sortino = (mean_ret / downside_std) * np.sqrt(252) if downside_std > 0 else -float('inf')
    calmar = (mean_ret * 252) / max_drawdown if max_drawdown > 0 else -float('inf')
    win_loss_pct = (wins / (wins + losses) * 100) if (wins + losses) > 0 else 0

    return {
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Win_Loss_Percentage": win_loss_pct,
        "Wins": wins,
        "Losses": losses
    }

# Backtesting function
def run_backtest(data: pd.DataFrame, params: dict, strategy: str):
    dataset = data.copy()

    # Calculate indicators with best parameters
    if strategy == 'RSI' or strategy == 'combined':
        rsi = ta.momentum.RSIIndicator(dataset.Close, window=params['rsi_window'])
        dataset["RSI"] = rsi.rsi()
    if strategy == 'BB' or strategy == 'combined':
        bb = ta.volatility.BollingerBands(dataset.Close, window=params['bb_window'], window_dev=params['bb_window_dev'])
        dataset["BB"] = bb.bollinger_mavg()
    if strategy == 'MACD' or strategy == 'combined':
        macd = ta.trend.MACD(dataset.Close, window_slow=params['macd_window_slow'], 
                             window_fast=params['macd_window_fast'], window_sign=params['macd_window_sign'])
        dataset["MACD"] = macd.macd()
        dataset["MACD_signal"] = macd.macd_signal()

    # Prepare LSTM data
    features = ["Close"]
    if strategy == 'RSI':
        features.append("RSI")
    elif strategy == 'BB':
        features.append("BB")
    elif strategy == 'MACD':
        features.extend(["MACD", "MACD_signal"])
    else:  # Combined
        features.extend(["RSI", "BB", "MACD", "MACD_signal"])

    X_train, y_train, X_test, y_test, scaler = prepare_lstm_data(dataset.dropna(), lookback=params['lookback'], features=features)
    model = train_lstm(X_train, y_train, lookback=params['lookback'])
    scaled_data = scaler.transform(dataset[features])
    X_full = [scaled_data[i - params['lookback']:i] for i in range(params['lookback'], len(scaled_data))]
    X_full = np.array(X_full)
    lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()

    dataset = dataset.iloc[params['lookback']:].reset_index(drop=True)
    dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
    dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    # Recalculate BB for signals after trimming dataset (if BB is used)
    if strategy == 'BB' or strategy == 'combined':
        bb = ta.volatility.BollingerBands(dataset.Close, window=params['bb_window'], window_dev=params['bb_window_dev'])

    # Define signals
    if strategy == 'RSI' or strategy == 'combined':
        dataset["RSI_BUY"] = dataset["RSI"] < params['rsi_lower']
        dataset["RSI_SELL"] = dataset["RSI"] > params['rsi_upper']
    if strategy == 'BB' or strategy == 'combined':
        dataset["BB_BUY"] = dataset.Close < bb.bollinger_lband()
        dataset["BB_SELL"] = dataset.Close > bb.bollinger_hband()
    if strategy == 'MACD' or strategy == 'combined':
        dataset["MACD_BUY"] = dataset["MACD"] > dataset["MACD_signal"]
        dataset["MACD_SELL"] = dataset["MACD"] < dataset["MACD_signal"]

    # Combine signals based on strategy
    if strategy == 'combined':
        dataset["BUY_SIGNAL"] = (dataset["LSTM_BUY"] | dataset["RSI_BUY"] | dataset["BB_BUY"] | dataset["MACD_BUY"])
        dataset["SELL_SIGNAL"] = (dataset["LSTM_SELL"] | dataset["RSI_SELL"] | dataset["BB_SELL"] | dataset["MACD_SELL"])
    else:
        dataset["BUY_SIGNAL"] = dataset[f"{strategy}_BUY"]
        dataset["SELL_SIGNAL"] = dataset[f"{strategy}_SELL"]

    dataset = dataset.dropna()

    # Trading simulation
    capital = 1000000
    com = 0.125 / 100
    portfolio_value = [capital]
    active_long_pos = None
    active_short_pos = None
    wins = 0
    losses = 0

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * params["n_shares"] * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * params["n_shares"] * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * params["n_shares"] * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * params["n_shares"] * (1 + com)
                capital += pnl
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * params["n_shares"] * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + params["take_profit"]),
                    "stop_loss": row.Close * (1 - params["stop_loss"])
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * params["n_shares"] * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - params["take_profit"]),
                "stop_loss": row.Close * (1 + params["stop_loss"])
            }

        long_value = row.Close * params["n_shares"] if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * params["n_shares"]) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    # Calculate final metrics
    final_metrics = calculate_metrics(portfolio_value, wins, losses)
    return final_metrics

# Run backtests for each strategy
def evaluate_all_strategies(data: pd.DataFrame):
    # RSI backtest
    rsi_metrics = run_backtest(data, RSI_PARAMS, 'RSI')
    print("RSI Backtest Metrics:", rsi_metrics)

    # BB backtest
    bb_metrics = run_backtest(data, BB_PARAMS, 'BB')
    print("BB Backtest Metrics:", bb_metrics)

    # MACD backtest
    macd_metrics = run_backtest(data, MACD_PARAMS, 'MACD')
    print("MACD Backtest Metrics:", macd_metrics)

    # Combined backtest (using MACD_PARAMS as base for lookback)
    combined_params = {
        'rsi_window': RSI_PARAMS['rsi_window'],
        'rsi_lower': RSI_PARAMS['rsi_lower'],
        'rsi_upper': RSI_PARAMS['rsi_upper'],
        'bb_window': BB_PARAMS['bb_window'],
        'bb_window_dev': BB_PARAMS['bb_window_dev'],
        'macd_window_slow': MACD_PARAMS['macd_window_slow'],
        'macd_window_fast': MACD_PARAMS['macd_window_fast'],
        'macd_window_sign': MACD_PARAMS['macd_window_sign'],
        'stop_loss': MACD_PARAMS['stop_loss'],  # Using MACD as base; adjust if desired
        'take_profit': MACD_PARAMS['take_profit'],
        'n_shares': MACD_PARAMS['n_shares'],
        'lookback': MACD_PARAMS['lookback']
    }
    combined_metrics = run_backtest(data, combined_params, 'combined')
    print("Combined Backtest Metrics:", combined_metrics)


### 6. Evaluación de Estrategias con Visualización

Esta función evalúa el desempeño de la estrategia de trading para cada indicador (RSI, Bollinger Bands y MACD) utilizando los parámetros óptimos predefinidos. En resumen, el código realiza lo siguiente:

* Preprocesamiento:
Se calcula una vez el conjunto de indicadores técnicos (RSI, BB y MACD) sobre el dataset completo, utilizando los parámetros óptimos definidos.

* Simulación y Backtesting:
Para cada estrategia, se ejecuta una simulación de trading que genera señales basadas en el indicador correspondiente, se simula la ejecución de operaciones (compras y ventas) y se actualiza el capital para obtener la evolución del portafolio.

* Cálculo de Métricas y Visualización:
Se imprimen el valor final del portafolio, los parámetros usados y las métricas de desempeño (Sharpe, Sortino, Calmar y Win/Loss Percentage). Además, se generan gráficas que muestran la evolución del portafolio comparada con el precio de cierre, facilitando la interpretación visual del rendimiento de cada estrategia.


In [ ]:
# Evaluate strategies with visualization
def evaluate_strategies(data: pd.DataFrame):
    # Preprocess data once with all indicators
    dataset = preprocess(
        data,
        rsi_window=RSI_PARAMS['rsi_window'],
        bb_window=BB_PARAMS['bb_window'],
        bb_window_dev=BB_PARAMS['bb_window_dev'],
        macd_window_slow=MACD_PARAMS['macd_window_slow'],
        macd_window_fast=MACD_PARAMS['macd_window_fast'],
        macd_window_sign=MACD_PARAMS['macd_window_sign']
    )

    # Run simulations for each strategy
    strategies = [
        ('RSI', RSI_PARAMS),
        ('BB', BB_PARAMS),
        ('MACD', MACD_PARAMS)
    ]

    for strategy_name, params in strategies:
        dataset_result, portfolio_value, final_metrics = simulate_trading(
            dataset.copy(),
            strategy=strategy_name,
            stop_loss=params['stop_loss'],
            take_profit=params['take_profit'],
            n_shares=params['n_shares'],
            lookback=params['lookback'] if strategy_name == 'MACD' else None
        )

        # Mostrar resultados
        print(f"\n--- {strategy_name} Strategy ---")
        print("Mejor valor del portafolio:", portfolio_value[-1])
        print("Mejores parámetros:", params)
        print("Métricas finales:")
        print(f"Sharpe Ratio: {final_metrics['Sharpe']:.2f}")
        print(f"Sortino Ratio: {final_metrics['Sortino']:.2f}")
        print(f"Calmar Ratio: {final_metrics['Calmar']:.2f}")
        print(f"Win/Loss Percentage: {final_metrics['Win_Loss_Percentage']:.2f}%")  # Fixed from 'Win_Loss_Percent'

        # Graficar
        plt.figure(figsize=(12, 6))
        plt.plot(portfolio_value, label="Portfolio Value")
        plt.legend()
        plt.twinx().plot(data.Close, c="orange", label="AAPL Close")
        plt.legend(loc="upper right")
        plt.title(f"{strategy_name} Strategy Performance")
        plt.show()

# Prepare and run the backtest
# Example: Load your data (replace with your actual data source)
dates = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")
data = pd.DataFrame({
    "Datetime": dates,
    "Close": np.random.normal(100, 10, len(dates))  # Random prices for demo
})

# Ensure data has the required columns
assert "Close" in data.columns and "Datetime" in data.columns, "DataFrame must have 'Close' and 'Datetime' columns"

# Run the evaluation
evaluate_strategies(data)

### 7. Configuración de Parámetros Óptimos y Evaluación de Estrategias con Visualización

Este código establece manualmente los parámetros óptimos para las estrategias basadas en RSI, Bollinger Bands y MACD mediante diccionarios. Además, implementa funciones de soporte para:

* Preparación y Normalización de Datos:
Se generan secuencias temporales a partir de los datos históricos y se normalizan las características (por defecto: Close, RSI, BB, MACD y MACD_signal) mediante MinMaxScaler.

* Entrenamiento del Modelo LSTM:
Se construye y entrena un modelo LSTM sencillo para predecir señales de trading (compra/venta) utilizando las secuencias generadas.

* Cálculo de Métricas de Desempeño:
Se calcula el Sharpe Ratio, Sortino Ratio, Calmar Ratio y el porcentaje de operaciones ganadoras (Win/Loss Percentage) a partir de la evolución del portafolio durante el backtesting.

* Preprocesamiento de Indicadores Técnicos:
Se calcula el RSI, las Bollinger Bands y el MACD a partir de los datos de precios, utilizando los parámetros óptimos definidos.

* Simulación de Trading (Backtesting):
Se simula la ejecución de operaciones (compras y ventas) basadas en las señales generadas por cada indicador (o una combinación de ellos), actualizando el capital y registrando los resultados de cada trade.

* Evaluación y Visualización:
Finalmente, se evalúan las estrategias para RSI, BB y MACD de forma individual, así como una estrategia combinada. Se imprimen los resultados (valor final del portafolio, parámetros utilizados y métricas de desempeño) y se generan gráficos que muestran la evolución del portafolio en comparación con el precio del activo.

In [ ]:
# Best parameters from your input
RSI_PARAMS = {
    'rsi_window': 46,
    'rsi_lower': 13,
    'rsi_upper': 65,
    'stop_loss': 0.06425843746848879,
    'take_profit': 0.02250238758183155,
    'n_shares': 1000,
    'lookback': 20
}

BB_PARAMS = {
    'bb_window': 26,
    'bb_window_dev': 1.6419616128479513,
    'stop_loss': 0.020183998738358935,
    'take_profit': 0.03515466268130313,
    'n_shares': 1000,
    'lookback': 20
}

MACD_PARAMS = {
    'macd_window_slow': 25,
    'macd_window_fast': 20,
    'macd_window_sign': 9,
    'stop_loss': 0.011148111728088396,
    'take_profit': 0.012560961178120339,
    'n_shares': 3000,
    'lookback': 20
}

# Supporting functions
def prepare_lstm_data(dataset, lookback=20, features=None):
    if features is None:
        features = ["Close", "RSI", "BB", "MACD", "MACD_signal"]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset[features])
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i - lookback:i])
        y.append(1 if scaled_data[i, 0] > scaled_data[i - 1, 0] else 0)
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test, scaler

def train_lstm(X_train, y_train, lookback, units=50):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    return model

# Your calculate_metrics function
def calculate_metrics(portfolio_value, trades):
    returns = np.diff(portfolio_value) / portfolio_value[:-1]

    # Ajuste de anualización: 78 intervalos de 5 minutos por día, 252 días al año
    annual_factor = np.sqrt(252 * 78)
    # Sharpe Ratio
    sharpe = np.mean(returns) / np.std(returns) * annual_factor if np.std(returns) > 0 else -float('inf')

    # Sortino Ratio
    downside_returns = returns[returns < 0]
    sortino = np.mean(returns) / np.std(downside_returns) * annual_factor if len(downside_returns) > 0 else -float('inf')

    # Calmar Ratio
    drawdowns = np.maximum.accumulate(portfolio_value) - portfolio_value
    max_drawdown = np.max(drawdowns)
    calmar = (portfolio_value[-1] - portfolio_value[0]) / max_drawdown if max_drawdown > 0 else -float('inf')
    
    # Win/Loss
    wins = sum(1 for t in trades if t["PnL"] > 0)
    losses = sum(1 for t in trades if t["PnL"] <= 0)
    win_loss = (wins / (wins + losses)) * 100 if (wins + losses) > 0 else 0
    
    return sharpe, sortino, calmar, win_loss

# Preprocessing function
def preprocess(data: pd.DataFrame, rsi_window: int, bb_window: int, bb_window_dev: float, 
               macd_window_slow: int, macd_window_fast: int, macd_window_sign: int) -> pd.DataFrame:
    dataset = data.copy()
    rsi = ta.momentum.RSIIndicator(dataset.Close, window=rsi_window)
    dataset['RSI'] = rsi.rsi()
    bb = ta.volatility.BollingerBands(dataset.Close, window=bb_window, window_dev=bb_window_dev)
    dataset['BB'] = bb.bollinger_mavg()
    dataset['BB_LOWER'] = bb.bollinger_lband()
    dataset['BB_UPPER'] = bb.bollinger_hband()
    macd = ta.trend.MACD(dataset.Close, window_slow=macd_window_slow, window_fast=macd_window_fast, window_sign=macd_window_sign)
    dataset['MACD'] = macd.macd()
    dataset['MACD_signal'] = macd.macd_signal()
    dataset = dataset.dropna()
    return dataset

# Trading simulation function adjusted for your trades format
def simulate_trading(dataset: pd.DataFrame, strategy: str, stop_loss: float, take_profit: float, n_shares: int, lookback: int = None) -> tuple:
    if strategy == 'MACD' and lookback is not None:
        X_train, y_train, _, _, scaler = prepare_lstm_data(dataset.dropna(), lookback=lookback, features=["Close", "MACD", "MACD_signal"])
        model = train_lstm(X_train, y_train, lookback=lookback)
        scaled_data = scaler.transform(dataset[["Close", "MACD", "MACD_signal"]])
        X_full = [scaled_data[i - lookback:i] for i in range(lookback, len(scaled_data))]
        X_full = np.array(X_full)
        lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()
        dataset = dataset.iloc[lookback:].reset_index(drop=True)
        dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
        dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    # Define signals
    if strategy == 'RSI':
        dataset['BUY_SIGNAL'] = dataset['RSI'] < RSI_PARAMS['rsi_lower']
        dataset['SELL_SIGNAL'] = dataset['RSI'] > RSI_PARAMS['rsi_upper']
    elif strategy == 'BB':
        dataset['BUY_SIGNAL'] = dataset.Close < dataset['BB_LOWER']
        dataset['SELL_SIGNAL'] = dataset.Close > dataset['BB_UPPER']
    elif strategy == 'MACD':
        dataset['BUY_SIGNAL'] = dataset['MACD'] > dataset['MACD_signal']
        dataset['SELL_SIGNAL'] = dataset['MACD'] < dataset['MACD_signal']
    else:
        raise ValueError("Strategy must be 'RSI', 'BB', or 'MACD'")

    dataset = dataset.dropna()
    capital = 1000000
    com = 0.125 / 100
    portfolio_value = [capital]
    trades = []  # Collect trades as dictionaries with "PnL"
    active_long_pos = None
    active_short_pos = None

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                trades.append({"PnL": pnl})
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * n_shares * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                trades.append({"PnL": pnl})
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                trades.append({"PnL": pnl})
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * n_shares * (1 + com)
                capital += pnl
                trades.append({"PnL": pnl})
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + take_profit),
                    "stop_loss": row.Close * (1 - stop_loss)
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * n_shares * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }

        long_value = row.Close * n_shares if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * n_shares) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    # Calculate metrics using your function
    sharpe, sortino, calmar, win_loss = calculate_metrics(portfolio_value, trades)
    final_metrics = {
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Win_Loss_Percentage": win_loss
    }
    return dataset, portfolio_value, final_metrics

# Evaluate strategies with visualization
def evaluate_strategies(data: pd.DataFrame):
    # Preprocess data once with all indicators
    dataset = preprocess(
        data,
        rsi_window=RSI_PARAMS['rsi_window'],
        bb_window=BB_PARAMS['bb_window'],
        bb_window_dev=BB_PARAMS['bb_window_dev'],
        macd_window_slow=MACD_PARAMS['macd_window_slow'],
        macd_window_fast=MACD_PARAMS['macd_window_fast'],
        macd_window_sign=MACD_PARAMS['macd_window_sign']
    )

    # Run simulations for each strategy
    strategies = [
        ('RSI', RSI_PARAMS),
        ('BB', BB_PARAMS),
        ('MACD', MACD_PARAMS)
    ]

    for strategy_name, params in strategies:
        dataset_result, portfolio_value, final_metrics = simulate_trading(
            dataset.copy(),
            strategy=strategy_name,
            stop_loss=params['stop_loss'],
            take_profit=params['take_profit'],
            n_shares=params['n_shares'],
            lookback=params['lookback'] if strategy_name == 'MACD' else None
        )

        # Mostrar resultados
        print(f"\n--- {strategy_name} Strategy ---")
        print("Mejor valor del portafolio:", portfolio_value[-1])
        print("Mejores parámetros:", params)
        print("Métricas finales:")
        print(f"Sharpe Ratio: {final_metrics['Sharpe']:.2f}")
        print(f"Sortino Ratio: {final_metrics['Sortino']:.2f}")
        print(f"Calmar Ratio: {final_metrics['Calmar']:.2f}")
        print(f"Win/Loss Percentage: {final_metrics['Win_Loss_Percentage']:.2f}%")

        # Graficar
        plt.figure(figsize=(12, 6))
        plt.plot(portfolio_value, label="Portfolio Value")
        plt.legend()
        plt.twinx().plot(data.Close, c="orange", label="AAPL Close")
        plt.legend(loc="upper right")
        plt.title(f"{strategy_name} Strategy Performance")
        plt.show()

# Prepare and run the backtest with sample 5-minute data
dates = pd.date_range(start="2023-01-01", end="2023-01-02", freq="5min")  # 1 day of 5-min data
data = pd.DataFrame({
    "Datetime": dates,
    "Close": np.random.normal(100, 1, len(dates))  # Small volatility for demo
})

# Ensure data has the required columns
assert "Close" in data.columns and "Datetime" in data.columns, "DataFrame must have 'Close' and 'Datetime' columns"

# Run the evaluation
evaluate_strategies(data)

### 8.  Evaluación de la Estrategia Combinada

Esta sección define los parámetros óptimos para una estrategia combinada que integra los indicadores RSI, Bollinger Bands y MACD. Primero, se establecen los valores predefinidos (COMBINED_PARAMS) que determinan las configuraciones para cada indicador y los parámetros generales de trading (stop loss, take profit, número de acciones y lookback).

Luego, se utilizan funciones de soporte para:

Preprocesar los datos: Calcular los indicadores técnicos (RSI, Bollinger Bands y MACD) sobre el dataset.
Preparar y entrenar un modelo LSTM: Generar secuencias normalizadas y entrenar un modelo que ayude a predecir señales de trading.
Simular el trading mediante backtesting: Se generan las señales individuales de cada indicador, se combinan para formar una señal final (requiriendo que al menos dos indicadores estén de acuerdo) y se simula la ejecución de operaciones, actualizando el capital y registrando cada trade.
Calcular métricas de desempeño: Se evalúa el rendimiento de la estrategia a través del Sharpe, Sortino, Calmar Ratio y el Win/Loss Percentage.
Visualizar los resultados: Se imprimen las métricas finales y se muestra una gráfica comparativa entre la evolución del portafolio y el precio del activo.
Esta estructura permite evaluar integralmente la eficacia de la estrategia combinada, comparando el desempeño de los diferentes indicadores y su sinergia en la toma de decisiones de trading.

In [ ]:
# Parámetros para la estrategia combinada
COMBINED_PARAMS = {
    'rsi_window': 14,  # Reducido para más sensibilidad
    'rsi_lower': 30,   # Más típico para compras
    'rsi_upper': 70,   # Más típico para ventas
    'bb_window': 20,   # Estándar para BB
    'bb_window_dev': 2.0,  # Más común
    'macd_window_slow': 26,
    'macd_window_fast': 12,
    'macd_window_sign': 9,
    'stop_loss': 0.02,  # Aumentado para más realismo
    'take_profit': 0.03,  # Aumentado para más realismo
    'n_shares': 100,   # Reducido para simplificar
    'lookback': 20
}

# Supporting functions
def prepare_lstm_data(dataset, lookback=20, features=None):
    if features is None:
        features = ["Close", "RSI", "MACD", "MACD_signal"]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset[features])
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i - lookback:i])
        y.append(1 if scaled_data[i, 0] > scaled_data[i - 1, 0] else 0)
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test, scaler

def train_lstm(X_train, y_train, lookback, units=50):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    return model

# Función calculate_metrics
def calculate_metrics(portfolio_value, trades):
    returns = np.diff(portfolio_value) / portfolio_value[:-1]
    annual_factor = np.sqrt(252 * 78)
    sharpe = np.mean(returns) / np.std(returns) * annual_factor if np.std(returns) > 0 else -float('inf')
    downside_returns = returns[returns < 0]
    sortino = np.mean(returns) / np.std(downside_returns) * annual_factor if len(downside_returns) > 0 else -float('inf')
    drawdowns = np.maximum.accumulate(portfolio_value) - portfolio_value
    max_drawdown = np.max(drawdowns)
    calmar = (portfolio_value[-1] - portfolio_value[0]) / max_drawdown if max_drawdown > 0 else -float('inf')
    wins = sum(1 for t in trades if t["PnL"] > 0)
    losses = sum(1 for t in trades if t["PnL"] <= 0)
    win_loss = (wins / (wins + losses)) * 100 if (wins + losses) > 0 else 0
    return sharpe, sortino, calmar, win_loss

# Preprocesamiento
def preprocess(data: pd.DataFrame, params: dict) -> pd.DataFrame:
    dataset = data.copy()
    rsi = ta.momentum.RSIIndicator(dataset.Close, window=params['rsi_window'])
    dataset['RSI'] = rsi.rsi()
    bb = ta.volatility.BollingerBands(dataset.Close, window=params['bb_window'], window_dev=params['bb_window_dev'])
    dataset['BB_LOWER'] = bb.bollinger_lband()
    dataset['BB_UPPER'] = bb.bollinger_hband()
    macd = ta.trend.MACD(dataset.Close, window_slow=params['macd_window_slow'], 
                         window_fast=params['macd_window_fast'], window_sign=params['macd_window_sign'])
    dataset['MACD'] = macd.macd()
    dataset['MACD_signal'] = macd.macd_signal()
    dataset = dataset.dropna()
    return dataset

# Simulación de trading
def simulate_trading(dataset: pd.DataFrame, params: dict) -> tuple:
    # LSTM para MACD
    X_train, y_train, _, _, scaler = prepare_lstm_data(dataset.dropna(), lookback=params['lookback'], 
                                                       features=["Close", "MACD", "MACD_signal"])
    model = train_lstm(X_train, y_train, lookback=params['lookback'])
    scaled_data = scaler.transform(dataset[["Close", "MACD", "MACD_signal"]])
    X_full = [scaled_data[i - params['lookback']:i] for i in range(params['lookback'], len(scaled_data))]
    X_full = np.array(X_full)
    lstm_preds = (model.predict(X_full, verbose=0) > 0.5).astype(int).flatten()
    dataset = dataset.iloc[params['lookback']:].reset_index(drop=True)
    dataset["LSTM_BUY"] = pd.Series(lstm_preds) == 1
    dataset["LSTM_SELL"] = pd.Series(lstm_preds) == 0

    # Señales individuales
    dataset["RSI_BUY"] = dataset["RSI"] < params['rsi_lower']
    dataset["RSI_SELL"] = dataset["RSI"] > params['rsi_upper']
    dataset["BB_BUY"] = dataset.Close < dataset["BB_LOWER"]
    dataset["BB_SELL"] = dataset.Close > dataset["BB_UPPER"]
    dataset["MACD_BUY"] = dataset["MACD"] > dataset["MACD_signal"]
    dataset["MACD_SELL"] = dataset["MACD"] < dataset["MACD_signal"]

    # Estrategia combinada: 2 o más indicadores
    dataset["BUY_SIGNAL"] = (dataset["RSI_BUY"].astype(int) + dataset["BB_BUY"].astype(int) + dataset["MACD_BUY"].astype(int)) >= 2
    dataset["SELL_SIGNAL"] = (dataset["RSI_SELL"].astype(int) + dataset["BB_SELL"].astype(int) + dataset["MACD_SELL"].astype(int)) >= 2

    capital = 1000000
    com = 0.125 / 100
    portfolio_value = [capital]
    trades = []
    active_long_pos = None
    active_short_pos = None

    for i, row in dataset.iterrows():
        if active_long_pos:
            if row.Close < active_long_pos["stop_loss"]:
                pnl = row.Close * params["n_shares"] * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                trades.append({"PnL": pnl})
                active_long_pos = None
            elif row.Close > active_long_pos["take_profit"]:
                pnl = row.Close * params["n_shares"] * (1 - com) - active_long_pos["cost"]
                capital += active_long_pos["cost"] + pnl
                trades.append({"PnL": pnl})
                active_long_pos = None

        if active_short_pos:
            if row.Close > active_short_pos["stop_loss"]:
                pnl = active_short_pos["revenue"] - row.Close * params["n_shares"] * (1 + com)
                capital += pnl
                trades.append({"PnL": pnl})
                active_short_pos = None
            elif row.Close < active_short_pos["take_profit"]:
                pnl = active_short_pos["revenue"] - row.Close * params["n_shares"] * (1 + com)
                capital += pnl
                trades.append({"PnL": pnl})
                active_short_pos = None

        if row["BUY_SIGNAL"] and active_long_pos is None and active_short_pos is None:
            cost = row.Close * params["n_shares"] * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_pos = {
                    "datetime": row.Datetime,
                    "cost": cost,
                    "take_profit": row.Close * (1 + params["take_profit"]),
                    "stop_loss": row.Close * (1 - params["stop_loss"])
                }

        if row["SELL_SIGNAL"] and active_short_pos is None and active_long_pos is None:
            revenue = row.Close * params["n_shares"] * (1 - com)
            capital += revenue
            active_short_pos = {
                "datetime": row.Datetime,
                "revenue": revenue,
                "take_profit": row.Close * (1 - params["take_profit"]),
                "stop_loss": row.Close * (1 + params["stop_loss"])
            }

        long_value = row.Close * params["n_shares"] if active_long_pos else 0
        short_value = (active_short_pos["revenue"] - row.Close * params["n_shares"]) if active_short_pos else 0
        portfolio_value.append(capital + long_value + short_value)

    sharpe, sortino, calmar, win_loss = calculate_metrics(portfolio_value, trades)
    final_metrics = {
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Calmar": calmar,
        "Win_Loss_Percentage": win_loss
    }
    return dataset, portfolio_value, final_metrics

# Evaluar la estrategia combinada
def evaluate_combined_strategy(data: pd.DataFrame):
    dataset = preprocess(data, COMBINED_PARAMS)
    dataset_result, portfolio_value, final_metrics = simulate_trading(dataset.copy(), COMBINED_PARAMS)

    print("\n--- Combined Strategy (RSI + BB + MACD) ---")
    print("Mejor valor del portafolio:", portfolio_value[-1])
    print("Mejores parámetros:", COMBINED_PARAMS)
    print("Métricas finales:")
    print(f"Sharpe Ratio: {final_metrics['Sharpe']:.2f}")
    print(f"Sortino Ratio: {final_metrics['Sortino']:.2f}")
    print(f"Calmar Ratio: {final_metrics['Calmar']:.2f}")
    print(f"Win/Loss Percentage: {final_metrics['Win_Loss_Percentage']:.2f}%")

    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_value, label="Portfolio Value")
    plt.legend()
    plt.twinx().plot(data.Close, c="orange", label="AAPL Close")
    plt.legend(loc="upper right")
    plt.title("Combined Strategy Performance")
    plt.show()

# Datos de prueba con más volatilidad
dates = pd.date_range(start="2023-01-01", end="2023-01-02", freq="5min")
data = pd.DataFrame({
    "Datetime": dates,
    "Close": 100 + np.cumsum(np.random.normal(0, 0.5, len(dates)))  # Movimiento más realista
})

assert "Close" in data.columns and "Datetime" in data.columns, "DataFrame must have 'Close' and 'Datetime' columns"
evaluate_combined_strategy(data)

### Conclusiones 


En conclusión, la estrategia combinada (RSI + BB + MACD) logró un rendimiento moderado con un incremento de capital hasta alrededor de 1,009,852. Las métricas de desempeño (especialmente Sharpe, Sortino y Calmar) son inusualmente altas, lo que sugiere una baja volatilidad de los retornos y drawdowns relativamente pequeños. No obstante, el porcentaje de aciertos (25%) indica que la mayoría de las operaciones fueron perdedoras, pero las pocas ganadoras obtuvieron beneficios suficientes para compensar las pérdidas. Estos resultados ponen de manifiesto la importancia de analizar en detalle la duración del periodo de prueba y el número de operaciones, así como de evaluar la estrategia bajo distintas condiciones de mercado para confirmar su robustez.